In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from scipy.sparse import csr_matrix
import random as rnd
import pickle
import sys


from datetime import datetime


from sklearn import preprocessing
from sklearn.model_selection import train_test_split




In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [3]:
def generate_random_array(median,std,size):
    output=[0]*size    
    for index in range(0,size//2):
        random_nr=rnd.uniform(-std,std)
        output[2*index]=median+random_nr
        output[2*index+1]=median-random_nr
    if(size % 2 != 0):
        output[size-1]=rnd.uniform(median-std,median+std)
    return output



In [4]:
traffic = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TrafficLabelling/combined.csv")
print(f'Number of Rows: {traffic.shape[0]}')
print(f'Number of Columns: {traffic.shape[1]}')
pd.set_option('display.max_colwidth', None)
traffic.head()

Number of Rows: 2660377
Number of Columns: 85


,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,...,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443,192.168.10.5,54865,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.5-104.16.28.216-55054-80-6,104.16.28.216,80,192.168.10.5,55054,6,7/7/2017 3:30,109,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,1.100917e+05,18348.62385,109.0,0.0,109.0,109.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.5-104.16.28.216-55055-80-6,104.16.28.216,80,192.168.10.5,55055,6,7/7/2017 3:30,52,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,2.307692e+05,38461.53846,52.0,0.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.16-104.17.241.25-46236-443-6,104.17.241.25,443,192.168.10.16,46236,6,7/7/2017 3:30,34,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,3.529412e+05,58823.52941,34.0,0.0,34.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,31,329,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.5-104.19.196.102-54863-443-6,104.19.196.102,443,192.168.10.5,54863,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [5]:
# Remove duplicate entries
traffic.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)

# reformat Timestamp to Unix time
# traffic[' Timestamp'] = pd.to_datetime(traffic[' Timestamp'])
# traffic[' Timestamp']= traffic[' Timestamp'].values.astype(np.int64) // 10 ** 9

# remove Timestamp column
traffic = traffic.drop(columns=[' Timestamp'])

# Remove constant columns
traffic = traffic.loc[:, traffic.apply(pd.Series.nunique) != 1]




In [6]:
# Remove column with NaN or Inf
traffic = traffic[~traffic.isin([np.nan, np.inf, -np.inf]).any(1)]

# remove Flow ID
traffic = traffic.drop(columns=['Flow ID'])

# reformat Label
traffic[' Label'] = traffic[' Label'].astype('category')
traffic[' Label'] = traffic[' Label'].cat.codes

# add new "is Attacker?" column
traffic[' is Attacker?'] = np.where(traffic[' Label'] == 0, 0, 1)

In [7]:
# combine ' Source IP' and ' Source Port'
traffic[' Source IP'] = traffic[' Source IP'] + ':' + traffic[' Source Port'].astype(str)
traffic[' Destination IP'] = traffic[' Destination IP'] + ':' + traffic[' Destination Port'].astype(str)
traffic.rename({' Source IP': ' SrcHost', ' Destination IP': ' DstHost'}, axis=1, inplace=True)

traffic = traffic.drop(columns=[' Source Port'])
traffic = traffic.drop(columns=[' Destination Port'])

traffic.head()


,SrcHost,DstHost,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,is Attacker?
0,104.16.207.165:443,192.168.10.5:54865,6,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,40,0,666666.666700,0.000000,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,9.0,6.0,0.0,40,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,104.16.28.216:80,192.168.10.5:55054,6,109,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,1.100917e+05,18348.62385,109.0,0.0,109.0,109.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,9174.311927,9174.311927,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,104.16.28.216:80,192.168.10.5:55055,6,52,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,2.307692e+05,38461.53846,52.0,0.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,19230.769230,19230.769230,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,104.17.241.25:443,192.168.10.16:46236,6,34,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,3.529412e+05,58823.52941,34.0,0.0,34.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,29411.764710,29411.764710,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,1,6,1,6,31,329,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,104.19.196.102:443,192.168.10.5:54863,6,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,40,0,666666.666700,0.000000,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,9.0,6.0,0.0,40,2,12,0,0,32,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [8]:

columns = list(traffic.columns)
columns


[' SrcHost',
 ' DstHost',
 ' Protocol',
 ' Flow Duration',
 ' Total Fwd Packets',
 ' Total Backward Packets',
 'Total Length of Fwd Packets',
 ' Total Length of Bwd Packets',
 ' Fwd Packet Length Max',
 ' Fwd Packet Length Min',
 ' Fwd Packet Length Mean',
 ' Fwd Packet Length Std',
 'Bwd Packet Length Max',
 ' Bwd Packet Length Min',
 ' Bwd Packet Length Mean',
 ' Bwd Packet Length Std',
 'Flow Bytes/s',
 ' Flow Packets/s',
 ' Flow IAT Mean',
 ' Flow IAT Std',
 ' Flow IAT Max',
 ' Flow IAT Min',
 'Fwd IAT Total',
 ' Fwd IAT Mean',
 ' Fwd IAT Std',
 ' Fwd IAT Max',
 ' Fwd IAT Min',
 'Bwd IAT Total',
 ' Bwd IAT Mean',
 ' Bwd IAT Std',
 ' Bwd IAT Max',
 ' Bwd IAT Min',
 'Fwd PSH Flags',
 ' Fwd URG Flags',
 ' Fwd Header Length',
 ' Bwd Header Length',
 'Fwd Packets/s',
 ' Bwd Packets/s',
 ' Min Packet Length',
 ' Max Packet Length',
 ' Packet Length Mean',
 ' Packet Length Std',
 ' Packet Length Variance',
 'FIN Flag Count',
 ' SYN Flag Count',
 ' RST Flag Count',
 ' PSH Flag Count',
 ' A

In [9]:
# create host dict {host: node_id}
src_hosts = set(traffic[' SrcHost'])
dst_hosts = set(traffic[' DstHost'])
hosts = list(src_hosts.union(dst_hosts))

                
host_dict = {hosts[i]: i for i in range(len(hosts))}   # dictionary of ip addresses { ip_addr: node_id }
print("Number of nodes: " + str(len(host_dict)))

Number of nodes: 435887


In [10]:
# adding a group_size column to the original dataframe
traffic['group_size'] = traffic.groupby([' SrcHost'])[' DstHost'].transform('size')


In [11]:
# define aggregation functions for all columns
aggregations = {col_name: "mean" for col_name in columns}
aggregations.pop(' Protocol', None)
aggregations.pop(' SrcHost', None)
aggregations.pop(' DstHost', None)
aggregations.pop(' Label', None)
aggregations.pop(' is Attacker?', None)

aggregations.update({' is Attacker?': lambda x: 1 if 1 in set(x) else 0})
print(aggregations)


{' Flow Duration': 'mean', ' Total Fwd Packets': 'mean', ' Total Backward Packets': 'mean', 'Total Length of Fwd Packets': 'mean', ' Total Length of Bwd Packets': 'mean', ' Fwd Packet Length Max': 'mean', ' Fwd Packet Length Min': 'mean', ' Fwd Packet Length Mean': 'mean', ' Fwd Packet Length Std': 'mean', 'Bwd Packet Length Max': 'mean', ' Bwd Packet Length Min': 'mean', ' Bwd Packet Length Mean': 'mean', ' Bwd Packet Length Std': 'mean', 'Flow Bytes/s': 'mean', ' Flow Packets/s': 'mean', ' Flow IAT Mean': 'mean', ' Flow IAT Std': 'mean', ' Flow IAT Max': 'mean', ' Flow IAT Min': 'mean', 'Fwd IAT Total': 'mean', ' Fwd IAT Mean': 'mean', ' Fwd IAT Std': 'mean', ' Fwd IAT Max': 'mean', ' Fwd IAT Min': 'mean', 'Bwd IAT Total': 'mean', ' Bwd IAT Mean': 'mean', ' Bwd IAT Std': 'mean', ' Bwd IAT Max': 'mean', ' Bwd IAT Min': 'mean', 'Fwd PSH Flags': 'mean', ' Fwd URG Flags': 'mean', ' Fwd Header Length': 'mean', ' Bwd Header Length': 'mean', 'Fwd Packets/s': 'mean', ' Bwd Packets/s': 'mean'

In [16]:
# low activity grouping
low_activities = traffic[traffic['group_size'] == 1]
low_activities = low_activities[low_activities[' is Attacker?'] == 0]
low_activities = low_activities.groupby(' SrcHost', as_index=False).agg(aggregations)
low_activities



,SrcHost,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,is Attacker?
0,0.0.0.0:0,36685713,2,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.054517,36700000.0,0.000000,36700000.0,36700000.0,36700000.0,36700000.0,0.000000,36700000.0,36700000.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.054517,0.000000,0.0,0.0,0.0,0.000000,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,2,0,0,0,-1,-1,0,0,0.0,0.0,0.0,0.0,36700000.0,0.0,36700000.0,36700000.0,0
1,1.193.219.21:443,648,3,0,258.0,0.0,258.0,0.0,86.0,148.956369,0.0,0.0,0.0,0.0,3.981481e+05,4629.629630,324.0,452.548340,644.0,4.0,648.0,324.0,452.548340,644.0,4.0,0.0,0.0,0.0,0.0,0.0,0,0,96,0,4629.629630,0.000000,0.0,258.0,64.5,129.000000,16641.0,0,0,0,0,1,0,0,0,0.0,86.0,86.0,0.0,96,3,258,0,0,61,-1,1,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.234.58.131:80,186262,2,0,6.0,0.0,6.0,0.0,3.0,4.242641,0.0,0.0,0.0,0.0,3.221269e+01,10.737563,186262.0,0.000000,186262.0,186262.0,186262.0,186262.0,0.000000,186262.0,186262.0,0.0,0.0,0.0,0.0,0.0,0,0,52,0,10.737563,0.000000,0.0,6.0,2.0,3.464102,12.0,0,0,0,0,1,0,0,0,0.0,3.0,3.0,0.0,52,2,6,0,0,140,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1.9.56.32:80,83,1,1,6.0,6.0,6.0,6.0,6.0,0.000000,6.0,6.0,6.0,0.0,1.445783e+05,24096.385542,83.0,0.000000,83.0,83.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,12048.192771,12048.192771,6.0,6.0,6.0,0.000000,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,1,6,1,6,980,253,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,101.102.235.200:443,2,1,1,6.0,6.0,6.0,6.0,6.0,0.000000,6.0,6.0,6.0,0.0,6.000000e+06,1000000.000000,2.0,0.000000,2.0,2.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,20,20,500000.000000,500000.000000,6.0,6.0,6.0,0.000000,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,1,6,1,6,279,251,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121365,98.158.54.101:443,67580,2,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,29.594555,67580.0,0.000000,67580.0,67580.0,67580.0,67580.0,0.000000,67580.0,67580.0,0.0,0.0,0.0,0.0,0.0,0,0,64,0,29.594555,0.000000,0.0,0.0,0.0,0.000000,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,64,2,0,0,0,62512,-1,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
121366,98.158.54.103:443,119023,2,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,16.803475,119023.0,0.000000,119023.0,119023.0,119023.0,119023.0,0.000000,119023.0,119023.0,0.0,0.0,0.0,0.0,0.0,0,0,64,0,16.803475,0.000000,0.0,0.0,0.0,0.000000,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,64,2,0,0,0,62512,-1,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
121367,98.158.54.104:443,2110,3,0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,1421.800948,1055.0,1429.769912,2066.0,44.0,2110.0,1055.0,1429.769912,2066.0,44.0,0.0,0

In [17]:
# attackers grouping
atk_activities = traffic[traffic[' is Attacker?'] == 1]
atk_activities = atk_activities.groupby(' SrcHost', as_index=False).agg(aggregations)
atk_activities


,SrcHost,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,is Attacker?
0,172.16.0.1:0,6.892114e+07,33.0,6.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.442271,5.123260e+06,6.206920e+06,1.789242e+07,1.017702e+06,6.893333e+07,6.351719e+06,7.254492e+06,1.789242e+07,1201029.5,23917195.0,2.736487e+06,3.247308e+06,6.797290e+06,675679.166667,0.0,0,0.0,0.0,0.373746,0.068525,0.0,0.0,0.000000,0.000000,0.000000e+00,0.0,0.0,0,0.0,0.0,0.0,0,0,0.166667,0.000000,0.000000,0.000,0.0,33.0,0.0,6.0,0.0,-1.000000,-1.000000,0.0,0.0,1786902.984,81093.990867,1.926828e+06,1.703254e+06,1.358333e+07,1.625716e+06,17150000.0,1.283333e+07,1
1,172.16.0.1:14099,4.025588e+06,4.0,3.0,28.0,5800.5,13.0,3.0,7.333333,5.131601,2190.0,0.0,966.750,878.894974,510405.728721,395.394360,1.005685e+06,2.011044e+06,4.023958e+06,1.150000e+01,4.025022e+06,1.007538e+06,2.012823e+06,4.023968e+06,66.0,5685.5,1.137100e+03,2.289932e+03,5.228000e+03,25.000000,0.0,0,86.0,66.0,132.005422,263.388938,3.0,2193.0,584.350000,822.620881,1.353410e+06,0.0,0.0,0,0.5,0.5,0.0,0,0,1.000000,649.544445,7.333333,966.750,86.0,4.0,28.0,3.0,5800.5,4224.000000,114.000000,3.0,20.0,990.500,0.000000,9.905000e+02,9.905000e+02,4.018902e+06,0.000000e+00,4018902.5,4.018902e+06,1
2,172.16.0.1:14100,4.024634e+06,4.0,3.0,28.0,5800.5,13.0,3.0,7.333333,5.131601,2190.0,0.0,966.750,878.894974,506889.999183,392.673050,1.005442e+06,2.010842e+06,4.023367e+06,3.000000e+00,4.023930e+06,1.007240e+06,2.012671e+06,4.023428e+06,3.0,5673.5,1.134700e+03,2.188398e+03,5.032000e+03,8.500000,0.0,0,86.0,66.0,131.098370,261.574680,3.0,2193.0,584.350000,822.620881,1.353410e+06,0.0,0.0,0,0.5,0.5,0.0,0,0,1.000000,649.544445,7.333333,966.750,86.0,4.0,28.0,3.0,5800.5,4224.000000,114.000000,3.0,20.0,499.500,0.000000,4.995000e+02,4.995000e+02,4.018400e+06,0.000000e+00,4018399.5,4.018400e+06,1
3,172.16.0.1:14101,4.023676e+06,4.0,2.0,28.0,5800.5,13.0,3.0,7.333333,5.131601,5067.5,0.0,1450.125,2435.912915,558187.175882,336.364832,1.005485e+06,2.010576e+06,4.022238e+06,3.200000e+01,4.022979e+06,1.006872e+06,2.011792e+06,4.022269e+06,212.0,5176.0,1.725333e+03,2.476014e+03,4.574500e+03,95.000000,0.0,0,86.0,46.0,144.334107,192.030725,3.0,5070.5,729.687500,1772.326013,6.300000e+06,0.0,0.0,0,0.5,0.5,0.0,0,0,0.500000,834.100000,7.333333,1450.125,86.0,4.0,28.0,2.0,5800.5,4224.000000,114.000000,3.0,20.0,498.500,0.000000,4.985000e+02,4.985000e+02,4.017970e+06,0.000000e+00,4017970.5,4.017970e+06,1
4,172.16.0.1:14102,4.022290e+06,4.0,2.5,28.0,5800.5,13.0,3.0,7.333333,5.131601,3650.0,0.0,1160.100,1511.254049,591586.277390,407.352972,1.005046e+06,2.009942e+06,4.021044e+06,2.500000e+00,4.021379e+06,1.006345e+06,2.011310e+06,4.021068e+06,2.5,4890.0,1.222500e+03,1.917825e+03,4.075000e+03,95.000000,0.0,0,86.0,56.0,152.951832,254.401140,3.0,3653.0,648.944445,1230.326493,3.027407e+06,0.0,0.0,0,0.5,0.5,0.0,0,0,0.500000,730.287500,7.333333,1160.100,86.0,4.0,28.0,2.5,58

In [18]:
basic_flows = traffic.groupby([' SrcHost', ' DstHost'], as_index=False)

basic_flows = basic_flows.agg(aggregations)
basic_flows

,SrcHost,DstHost,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,is Attacker?
0,0.0.0.0:0,224.0.0.22:0,36685713.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.054517,36700000.0,0.0000,36700000.0,36700000.0,36700000.0,36700000.0,0.0,36700000.0,36700000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054517,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,36700000.0,0.0,36700000.0,36700000.0,0
1,1.1.70.73:80,192.168.10.15:52772,344.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,34883.720930,5813.953488,344.0,0.0000,344.0,344.0,344.0,344.0,0.0,344.0,344.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,5813.953488,0.000000,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,6.0,0.0,40.0,2.0,12.0,0.0,0.0,4380.0,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.1.70.73:80,192.168.10.17:48301,315.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,6349.206349,315.0,0.0000,315.0,315.0,315.0,315.0,0.0,315.0,315.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,6349.206349,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,2.0,0.0,0.0,0.0,4380.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1.1.70.73:80,192.168.10.19:35264,179391.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,16.723247,89695.5,126775.0535,179339.0,52.0,179391.0,179391.0,0.0,179391.0,179391.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,32.0,11.148831,5.574416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,2.0,0.0,1.0,0.0,4677.0,240.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1.1.70.73:80,192.168.10.19:35275,417.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4796.163070,417.0,0.0000,417.0,417.0,417.0,417.0,0.0,417.0,417.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,4796.163070,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,2.0,0.0,0.0,0.0,4380.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1382932,99.192.248.32:80,192.168.10.5:49372,55.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,218181.818182,36363.636364,55.0,0.0000,55.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,18181.818182,18181.818182,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,6.0,6.0,20.0,1.0,6.0,1.0,6.0,46.0,256.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1382933,99.192.248.32:80,192.168.10.5:49419,45.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,266666.666667,44444.444444,45.0,0.0000,45.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,22222.222222,22222.222222,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0

In [19]:
#activity_flows
activity_flows = basic_flows.groupby([' SrcHost'], as_index=False)
activity_flows = activity_flows.agg(aggregations)





In [20]:
# reformat Source IP
activity_flows[" SrcHost"] = activity_flows[" SrcHost"].map(host_dict)
print(activity_flows)

atkers = activity_flows.groupby([' is Attacker?']).get_group(1)
non_atkers = activity_flows.groupby([' is Attacker?']).get_group(0)
print("Number of attackers: " + str(len(atkers)))
print("Number of non_atkers " + str(len(non_atkers)))

         SrcHost   Flow Duration  ...    Idle Min   is Attacker?
0         403376    3.668571e+07  ...  36700000.0              0
1         419087    4.511675e+04  ...         0.0              0
2         322118    6.480000e+02  ...         0.0              0
3         352494    1.038322e+05  ...         0.0              0
4         174039    4.217540e+04  ...         0.0              0
...          ...             ...  ...         ...            ...
425242    133478    4.627100e+04  ...         0.0              0
425243    164261    4.732533e+04  ...         0.0              0
425244    105379    1.156000e+03  ...         0.0              0
425245    390208    8.496417e+03  ...         0.0              0
425246    152565    3.000000e+00  ...         0.0              0

[425247 rows x 71 columns]
Number of attackers: 49643
Number of non_atkers 375604


In [21]:
# create low profiles dataframe
num_hosts = len(host_dict)
num_src_hosts = len(activity_flows[' SrcHost'])
num_noflow_hosts = num_hosts - num_src_hosts


no_flow_prof_columns = columns[3:-3]
print(no_flow_prof_columns)
num_low_hosts = round(1*num_noflow_hosts)   #### adjust benign/attackers ratio here
low_activities_prof = np.zeros((num_low_hosts, len(no_flow_prof_columns)))
for col, attr in enumerate(no_flow_prof_columns):
  median = low_activities[attr].median()
  std = np.std(low_activities[attr])
  low_activities_prof[:, col] = generate_random_array(median, std, num_low_hosts)


low_profile_df = pd.DataFrame(low_activities_prof, columns = no_flow_prof_columns)
low_profile_df[' is Attacker?'] = [0 for i in range(num_low_hosts)]
low_profile_df[' SrcHost'] = [num_src_hosts + i for i in range(num_low_hosts)]
low_profile_df



[' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Fwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' ECE Flag Count', ' Down/Up R

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,is Attacker?,SrcHost
0,-7.780621e+06,-17.047728,27.880900,-7785.824595,-31094.045690,-249.087896,-33.469960,104.209887,47.689250,425.180161,124.742730,158.645194,-28.030891,8.736398e+05,-85805.522547,-665497.972957,1.152790e+06,-4.184640e+06,-174737.158457,2.605234e+07,6.113037e+05,3.013128e+05,-2.815871e+05,-8739.835716,1.828260e+07,1.019945e+06,1.102229e+06,-1.140497e+06,40267.897550,0.019174,-0.001493,-4.405831e+06,8.699949e+05,53150.522660,6308.724883,37.867590,-591.222850,134.178999,93.794238,-77161.879698,-0.089542,-0.040249,0.015899,-0.367247,-0.087546,-0.079919,0.006821,-0.000686,1.276470,139.298782,111.853600,155.106981,-1.312225e+07,14.366445,-845.815534,-16.504249,-49811.210840,-3964.948171,-6563.999473,2.749750,4.867341e+05,132691.513274,96800.247265,-63028.539500,-39825.692848,2.701703e+06,543430.461727,-3.885170e+06,0,425247
1,7.828527e+06,21.047728,-23.880900,7929.824595,31486.045690,321.087896,99.469960,-34.209887,-47.689250,-227.180161,41.257270,35.354806,28.030891,-8.452604e+05,86140.138575,681414.639623,-1.125349e+06,4.232446e+06,174743.158457,-2.605233e+07,-6.112977e+05,-3.013128e+05,2.815931e+05,8745.835716,-1.828259e+07,-1.019937e+06,-1.102229e+06,1.140505e+06,-40259.897550,-0.019174,0.001493,4.405959e+06,-8.698669e+05,-52983.028993,-6143.469771,28.132410,789.222850,-6.978999,-28.067531,79321.879698,0.089542,0.040249,-0.015899,0.367247,0.087546,0.079919,-0.006821,0.000686,0.723530,16.701218,-41.853600,38.893019,1.312238e+07,-10.366445,989.815534,20.504249,50203.210840,3962.948171,6561.999473,-0.749750,-4.866701e+05,-132691.513274,-96800.247265,63028.539500,39825.692848,-2.701703e+06,-543430.461727,3.885170e+06,0,425248
2,2.267731e+07,3.038018,19.616791,-3010.696694,-49748.112568,219.697434,-30.738942,21.770186,6.432065,579.828837,127.867957,-60.882209,-188.920769,5.594393e+05,35541.292967,-631839.307043,1.202811e+06,8.041164e+05,-155441.714312,-4.827871e+06,1.165573e+06,5.360119e+05,4.958639e+06,-42585.926061,-2.547647e+07,7.395006e+04,8.640921e+05,2.837274e+06,-74573.076573,-0.011392,0.006139,1.034125e+07,8.681517e+05,-80847.541651,-9325.186830,9.439072,-58.485782,140.452084,-51.023571,68856.998545,0.043051,0.022633,-0.021667,0.345453,0.008326,0.019959,0.012066,0.031867,1.323644,110.384101,118.771591,250.187082,5.075979e+06,14.685878,-3424.968492,-25.285550,-7475.297527,-3428.338027,-2745.678111,-2.071209,-8.217895e+05,-141414.138558,-95165.782861,-565428.181483,336891.178811,-2.568624e+05,-411491.962520,8.423268e+05,0,425249
3,-2.262941e+07,0.961982,-15.616791,3154.696694,50140.112568,-147.697434,96.738942,48.229814,-6.432065,-381.828837,38.132043,254.882209,188.920769,-5.310600e+05,-35206.676939,647755.973709,-1.175370e+06,-7.563104e+05,155447.714312,4.827877e+06,-1.165567e+06,-5.360119e+05,-4.958633e+06,42591.926061,2.547648e+07,-7.394206e+04,-8.640921e+05,-2.837266e+06,74581.076573,0.011

In [22]:
# create attacker profiles dataframe

num_atk_hosts = num_noflow_hosts-num_low_hosts

atker_prof = np.zeros((num_atk_hosts, len(no_flow_prof_columns)))
for col, attr in enumerate(no_flow_prof_columns):
  median = atk_activities[attr].median()
  std = np.std(atk_activities[attr])
  atker_prof[:, col] = generate_random_array(median, std, num_atk_hosts)


atker_profile_df = pd.DataFrame(atker_prof, columns = no_flow_prof_columns)
atker_profile_df[' is Attacker?'] = [1 for i in range(num_atk_hosts)]
atker_profile_df[' SrcHost'] = [num_src_ips+num_low_ips + i for i in range(num_atk_hosts)]
atker_profile_df


,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,is Attacker?,SrcHost


In [23]:
# combines all the dataframes
activity_flows_all = activity_flows.append(low_profile_df, ignore_index=True)
activity_flows_all = activity_flows_all.append(atker_profile_df, ignore_index=True)



In [24]:

atkers = activity_flows_all.groupby([' is Attacker?']).get_group(1)
non_atkers = activity_flows_all.groupby([' is Attacker?']).get_group(0)
print("Number of attackers: " + str(len(atkers)))
print("Number of non_atkers " + str(len(non_atkers)))


Number of attackers: 49643
Number of non_atkers 386244


In [25]:
activity_flows_all = activity_flows_all.sort_values(by=' SrcHost', ignore_index=True)
activity_flows_all

,SrcHost,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,is Attacker?
0,0.0,5.834990e+05,3.00000,5.000000,26.000000,11607.000000,20.000000,0.000000,8.666667,10.263203,7215.000000,0.000000,2321.400000,3327.769794,1.993662e+04,13.710392,8.335700e+04,2.200927e+05,5.824800e+05,3.000000,7.810000e+02,3.905000e+02,5.296230e+02,7.650000e+02,16.000000,5.834440e+05,1.458610e+05,2.910795e+05,5.824800e+05,3.000000,0.000000,0.000000,7.200000e+01,1.120000e+02,5.141397,8.568995,0.00000,7215.00000,1292.555556,2650.585648,7.025604e+06,0.00000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1454.125000,8.666667,2321.400000,7.200000e+01,3.000000,26.000000,5.000000,11607.000000,8192.000000,229.000000,2.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000,1.0
1,1.0,1.536738e+07,160.00000,223.500000,3228.000000,435408.500000,552.500000,3.000000,17.281809,66.517217,2148.000000,0.000000,974.068233,408.876740,1.747188e+05,11206.458825,2.017415e+04,3.129993e+05,4.999352e+06,24.000000,1.535007e+07,4.863700e+04,4.861033e+05,5.000043e+06,25.500000,1.535000e+07,3.443093e+04,4.098351e+05,5.000000e+06,0.500000,0.000000,0.000000,3.206000e+03,4.488000e+03,11199.186899,7.271927,3.00000,2163.50000,579.477941,576.386396,6.501891e+05,0.00000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.500000,582.270070,17.281809,974.068233,3.206000e+03,160.000000,3228.000000,223.500000,435408.500000,36853.000000,80.000000,159.000000,20.000000,121396.833350,191207.728850,342184.500000,10987.000000,4.998172e+06,1940.260055,4.999310e+06,4995932.000,0.0
2,2.0,4.815000e+04,2.00000,2.000000,64.000000,96.000000,32.000000,32.000000,32.000000,0.000000,48.000000,48.000000,48.000000,0.000000,3.322949e+03,83.073728,1.605000e+04,2.775612e+04,4.810000e+04,3.000000,3.000000e+00,3.000000e+00,0.000000e+00,3.000000e+00,3.000000,4.700000e+01,4.700000e+01,0.000000e+00,4.700000e+01,47.000000,0.000000,0.000000,8.000000e+01,6.400000e+01,41.536864,41.536864,32.00000,48.00000,38.400000,8.763561,7.680000e+01,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,48.000000,32.000000,48.000000,8.000000e+01,2.000000,64.000000,2.000000,96.000000,-1.000000,-1.000000,1.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000,0.0
3,3.0,2.402768e+06,4.00000,2.000000,28.000000,5803.500000,13.000000,3.000000,7.333333,5.131601,5797.500000,0.000000,1450.875000,2897.750345,3.558767e+03,2.927498,5.325008e+05,1.125670e+06,2.401656e+06,9.500000,1.584942e+06,3.963530e+05,7.921646e+05,1.584391e+06,49.000000,8.182865e+05,2.727622e+05,4.719179e+05,8.176860e+05,91.000000,0.000000,0.000000,8.600000e+01,4.600000e+01,1.705445,1.222052,3.00000,5800.50000,730.062500,2048.768767,8.400000e+06,0.00000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.500000,834.528571,7.333333,1450.875000,8.600000e+01

In [26]:
# (1) SELECT | ALL features
X = activity_flows_all.iloc[:, 1:(activity_flows_all.shape[1]-1)]
Y = activity_flows_all.iloc[:, -1]


In [27]:
# create hypergraph
# connection = traffic.groupby([' SrcHost', ' DstHost'], as_index=False)
# connection = connection.agg({' Flow Duration' : 'count'})
src_host_ids = [host_dict[host] for host in basic_flows[' SrcHost']]
dst_host_ids = [host_dict[host] for host in basic_flows[' DstHost']]
edges = list(zip(src_host_ids, dst_host_ids))
hypergraph = {i: edges[i] for i in range(len(edges))}

In [28]:
# # generate features
features = X
features = np.nan_to_num(features)
features = preprocessing.normalize(features)
features = csr_matrix(features)

# generate features from feature selection set
# X = X.fillna(0)
# selected_features = selectFeatures(X, Y)
# print(f'Selected Features "from All": {selected_features}')
# features = X[selected_features]
# print("Shape of features: ", features.shape)

# features = preprocessing.normalize(features)
# features = csr_matrix(features)


In [29]:
# generate labels
labels = Y

In [30]:
# train/test split

splits = {}
V = len(hosts)
node_sorted = list(range(V))
rnd.shuffle(node_sorted)
train_size = round(0.8*V)
test_size = V - train_size
train_set, test_set = train_test_split(node_sorted, train_size=train_size, test_size=test_size)



splits['train'] = train_set
splits['test'] = test_set


In [31]:
train_atk_count = 0
for idx, train_id in enumerate(train_set):
    if labels[train_id] == 1:
      train_atk_count += 1

test_atk_count = 0
for idx, test_id in enumerate(test_set):
    if labels[test_id] == 1:
      test_atk_count += 1

print("Number of atker in train set: ", train_atk_count)
print("Number of atker in test set: ", test_atk_count)

Number of atker in train set:  39612
Number of atker in test set:  10031


In [34]:
# generate files
with open("hypergraph.pickle", "wb") as hypergraph_out:
    pickle.dump(hypergraph, hypergraph_out)
    hypergraph_out.close()
with open("features.pickle", "wb") as features_out:
    pickle.dump(features, features_out)
    features_out.close()
with open("labels.pickle", "wb") as labels_out:
    pickle.dump(labels, labels_out)
    labels_out.close()
with open("1.pickle", "wb") as splits_out:
    pickle.dump(splits, splits_out)
    splits_out.close()